In [155]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import NeuralNetwork as NN
import Layer
import importlib
importlib.reload(NN)
importlib.reload(Layer)

<module 'Layer' from '/Users/jorchard/Dropbox/research/peBogacz/python/Layer.py'>

In [156]:
net = NN.NeuralNetwork()

In [157]:
l = Layer.InputPELayer(n=5)
net.AddLayer(l)
net.AddLayer(Layer.PELayer(n=3))
net.AddLayer(Layer.PELayer(n=4))

In [158]:
net.W[0][0,0] = 1.
net.layers[0].e[0] = 1.

In [159]:
net.W

[tensor([[ 1.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]]), tensor([[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]])]

In [160]:
net.layers[0].DisplayState()
net.layers[0].DisplayError()

tensor([ 0.,  0.,  0.,  0.,  0.])
tensor([ 1.,  0.,  0.,  0.,  0.])


In [161]:
net.Integrate()

1
2


In [162]:
net.layers[1].DisplayState()

tensor([ 0.,  0.,  0.])


In [163]:
net.layers[1].dvdt

tensor([ 1.,  0.,  0.])

In [164]:
net.Step(dt=0.001)

0
1
2


In [165]:
net.layers[1].v

tensor(1.00000e-03 *
       [ 1.0000,  0.0000,  0.0000])

In [166]:
net.layers[1].v

tensor(1.00000e-03 *
       [ 1.0000,  0.0000,  0.0000])

In [167]:
net.layers[1].Output_Down()

tensor([ 0.5002,  0.5000,  0.5000])